# Basket Analysis- Apriori, FP Growth & ECLAT

In [1]:
# Importar librerias 

import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [2]:
#Importar dataset de transaciones

In [3]:
Ratail_df= pd.read_excel("Online_Retail.xlsx")

In [4]:
Ratail_df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [6]:
Ratail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [9]:
# Conociendo los valores nulos
Ratail_df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [10]:
Ratail_df['Country'].value_counts()

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

In [11]:
#Filtrando por españa
Spain_Ratail_df=Ratail_df[Ratail_df['Country']=='Spain']

In [12]:
Spain_Ratail_df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
6421,536944,22383,LUNCH BAG SUKI DESIGN,70,2010-12-03 12:20:00,1.65,12557.0,Spain
6422,536944,22384,LUNCH BAG PINK POLKADOT,100,2010-12-03 12:20:00,1.45,12557.0,Spain
6423,536944,20727,LUNCH BAG BLACK SKULL.,60,2010-12-03 12:20:00,1.65,12557.0,Spain
6424,536944,20725,LUNCH BAG RED RETROSPOT,70,2010-12-03 12:20:00,1.65,12557.0,Spain
6425,536944,20728,LUNCH BAG CARS BLUE,100,2010-12-03 12:20:00,1.45,12557.0,Spain
21565,538095,22504,CABIN BAG VINTAGE RETROSPOT,1,2010-12-09 14:55:00,29.95,17097.0,Spain
21566,538095,22212,FOUR HOOK WHITE LOVEBIRDS,1,2010-12-09 14:55:00,2.10,17097.0,Spain
21567,538095,22953,BIRTHDAY PARTY CORDON BARRIER TAPE,1,2010-12-09 14:55:00,1.25,17097.0,Spain
21568,538095,84818,DANISH ROSE PHOTO FRAME,1,2010-12-09 14:55:00,2.55,17097.0,Spain
21569,538095,21114,LAVENDER SCENTED FABRIC HEART,5,2010-12-09 14:55:00,1.25,17097.0,Spain


In [13]:
# Número de transaciones hechas
Spain_Ratail_df['InvoiceNo'].nunique()


105

In [15]:
# haciendo la columna de descripcion Proper case in pandas es llamado title


Spain_Ratail_df['Description']=Spain_Ratail_df['Description'].str.title()

<ipython-input-15-540f3144c515>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Spain_Ratail_df['Description']=Spain_Ratail_df['Description'].str.title()


In [16]:
#Tocando el dataset
Spain_Ratail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
6421,536944,22383,Lunch Bag Suki Design,70,2010-12-03 12:20:00,1.65,12557.0,Spain
6422,536944,22384,Lunch Bag Pink Polkadot,100,2010-12-03 12:20:00,1.45,12557.0,Spain
6423,536944,20727,Lunch Bag Black Skull.,60,2010-12-03 12:20:00,1.65,12557.0,Spain
6424,536944,20725,Lunch Bag Red Retrospot,70,2010-12-03 12:20:00,1.65,12557.0,Spain
6425,536944,20728,Lunch Bag Cars Blue,100,2010-12-03 12:20:00,1.45,12557.0,Spain


In [17]:
# Creando las transaciones de los valores unicos.

transactions=[]
for item in Spain_Ratail_df['InvoiceNo'].unique():
    lst=list(set(Spain_Ratail_df[Spain_Ratail_df['InvoiceNo']==item]['Description']))
    transactions.append(lst)

transactions[0:5]

[['Lunch Bag Suki  Design ',
  'Lunch Bag  Black Skull.',
  'Lunch Bag Pink Polkadot',
  'Lunch Bag Red Retrospot',
  'Lunch Bag Cars Blue'],
 ['Folkart Zinc Heart Christmas Dec',
  'Heart Decoration Painted Zinc ',
  'Wooden Tree Christmas Scandinavian',
  'Lavender Scented Fabric Heart',
  'Alarm Clock Bakelike Green',
  'Blue Coat Rack Paris Fashion',
  'Cabin Bag Vintage Retrospot',
  'Party Bunting',
  'Wooden Heart Christmas Scandinavian',
  'Circus Parade Lunch Box ',
  'Hearts Gift Tape',
  'Emergency First Aid Tin ',
  'Rocking Horse Green Christmas ',
  'Birthday Party Cordon Barrier Tape',
  'Set Of 20 Vintage Christmas Napkins',
  'Red Coat Rack Paris Fashion',
  'Glitter Hanging Butterfly String',
  'Love Heart Napkin Box ',
  'Heart Of Wicker Small',
  'Star Wooden Christmas Decoration',
  'Set Of 2 Tins Vintage Bathroom ',
  'White Hanging Heart T-Light Holder',
  'Stars Gift Tape ',
  'Four Hook  White Lovebirds',
  'Photo Frame 3 Classic Hanging',
  'Cupid Design Scent

<h4> <h4> Utilizando el  Apriori Basket Analisis

In [18]:
from efficient_apriori import apriori

In [19]:
# our min support is 7, but it has to be expressed as a percentage for efficient-apriori
min_support = 7/len(transactions) 

In [20]:
min_support

0.06666666666666667

In [21]:
min_confidence = 0

In [22]:
#Creando las reglas

itemsets, rules = apriori(transactions, min_support=min_support, min_confidence=min_confidence)

In [23]:
itemsets

{1: {('Lunch Bag Pink Polkadot',): 10,
  ('Lunch Bag Red Retrospot',): 8,
  ('Alarm Clock Bakelike Green',): 8,
  ('Party Bunting',): 8,
  ('White Hanging Heart T-Light Holder',): 11,
  ('Classic Metal Birdcage Plant Holder',): 11,
  ('Postage',): 62,
  ('Regency Cakestand 3 Tier',): 25,
  ('Plasters In Tin Circus Parade ',): 7,
  ('Plasters In Tin Skulls',): 11,
  ('Set Of 3 Heart Cookie Cutters',): 7,
  ("Poppy'S Playhouse Kitchen",): 7,
  ('Set Of 36 Paisley Flower Doilies',): 8,
  ('Spaceboy Lunch Box ',): 12,
  ("Poppy'S Playhouse Bedroom ",): 8,
  ('Set Of 72 Retrospot Paper  Doilies',): 10,
  ('Paper Bunting Retrospot',): 7,
  ('Strawberry Ceramic Trinket Box',): 7,
  ('Set Of 36 Teatime Paper Doilies',): 7,
  ('Pack Of 60 Pink Paisley Cake Cases',): 8,
  ('Red Retrospot Cake Stand',): 10,
  ('Set Of 3 Cake Tins Pantry Design ',): 7,
  ('Set/10 Pink Polkadot Party Candles',): 7,
  ('Jam Making Set With Jars',): 16,
  ('Plasters In Tin Woodland Animals',): 9,
  ('6 Ribbons Rustic

In [24]:
print(rules)

[{Assorted Colour Bird Ornament} -> {6 Ribbons Rustic Charm}, {6 Ribbons Rustic Charm} -> {Assorted Colour Bird Ornament}, {Postage} -> {6 Ribbons Rustic Charm}, {6 Ribbons Rustic Charm} -> {Postage}, {Postage} -> {Assorted Colour Bird Ornament}, {Assorted Colour Bird Ornament} -> {Postage}, {Postage} -> {Classic Metal Birdcage Plant Holder}, {Classic Metal Birdcage Plant Holder} -> {Postage}, {Postage} -> {Doormat Spotty Home Sweet Home}, {Doormat Spotty Home Sweet Home} -> {Postage}, {Postage} -> {Jam Making Set With Jars}, {Jam Making Set With Jars} -> {Postage}, {Postage} -> {Pack Of 72 Retrospot Cake Cases}, {Pack Of 72 Retrospot Cake Cases} -> {Postage}, {Postage} -> {Plasters In Tin Skulls}, {Plasters In Tin Skulls} -> {Postage}, {Poppy'S Playhouse Kitchen} -> {Poppy'S Playhouse Bedroom }, {Poppy'S Playhouse Bedroom } -> {Poppy'S Playhouse Kitchen}, {Postage} -> {Poppy'S Playhouse Bedroom }, {Poppy'S Playhouse Bedroom } -> {Postage}, {Regency Cakestand 3 Tier} -> {Postage}, {Pos

In [25]:
for rule in rules:
    print(rule)

{Assorted Colour Bird Ornament} -> {6 Ribbons Rustic Charm} (conf: 0.750, supp: 0.086, lift: 5.250, conv: 3.429)
{6 Ribbons Rustic Charm} -> {Assorted Colour Bird Ornament} (conf: 0.600, supp: 0.086, lift: 5.250, conv: 2.214)
{Postage} -> {6 Ribbons Rustic Charm} (conf: 0.210, supp: 0.124, lift: 1.468, conv: 1.085)
{6 Ribbons Rustic Charm} -> {Postage} (conf: 0.867, supp: 0.124, lift: 1.468, conv: 3.071)
{Postage} -> {Assorted Colour Bird Ornament} (conf: 0.177, supp: 0.105, lift: 1.552, conv: 1.077)
{Assorted Colour Bird Ornament} -> {Postage} (conf: 0.917, supp: 0.105, lift: 1.552, conv: 4.914)
{Postage} -> {Classic Metal Birdcage Plant Holder} (conf: 0.145, supp: 0.086, lift: 1.386, conv: 1.047)
{Classic Metal Birdcage Plant Holder} -> {Postage} (conf: 0.818, supp: 0.086, lift: 1.386, conv: 2.252)
{Postage} -> {Doormat Spotty Home Sweet Home} (conf: 0.113, supp: 0.067, lift: 1.694, conv: 1.052)
{Doormat Spotty Home Sweet Home} -> {Postage} (conf: 1.000, supp: 0.067, lift: 1.694, con

<h4> Utilizando el FP Growth Basket Analisis

In [26]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [27]:
# instantiate a transaction encoder
my_transactionencoder = TransactionEncoder()

In [28]:
# fit the transaction encoder using the list of transaction tuples
my_transactionencoder.fit(transactions)

TransactionEncoder()

In [29]:
# transform the list of transaction tuples into an array of encoded transactions
encoded_transactions = my_transactionencoder.transform(transactions)

In [30]:
# convert the array of encoded transactions into a dataframe
encoded_transactions_df = pd.DataFrame(encoded_transactions, columns=my_transactionencoder.columns_)
encoded_transactions_df.head()

,50'S Christmas Gift Bag Large,Red Spot Gift Bag Large,Set 2 Tea Towels I Love London,Spaceboy Baby Gift Set,10 Colour Spaceboy Pen,12 Egg House Painted Wood,12 Message Cards With Envelopes,12 Pencil Small Tube Woodland,20 Dolly Pegs Retrospot,200 Red + White Bendy Straws,...,Wrap Red Apples,Wrap Red Vintage Doily,Wrap Suki And Friends,"Wrap, Billboard Fonts Design",Yellow Coat Rack Paris Fashion,Yellow Flowers Felt Handbag Kit,Zinc Folkart Sleigh Bells,Zinc Metal Heart Decoration,Zinc T-Light Holder Star Large,Zinc T-Light Holder Stars Small
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [32]:
# our min support is 7, but it has to be expressed as a percentage for mlxtend
min_support = 7/len(transactions) 

In [33]:
# compute the frequent itemsets using fpgriowth from mlxtend
from mlxtend.frequent_patterns.fpgrowth import fpgrowth
frequent_itemsets = fpgrowth(encoded_transactions_df, min_support=min_support, use_colnames = True)

In [34]:
# print the frequent itemsets
frequent_itemsets

,support,itemsets
0,0.095238,(Lunch Bag Pink Polkadot)
1,0.076190,(Lunch Bag Red Retrospot)
2,0.104762,(White Hanging Heart T-Light Holder)
3,0.076190,(Alarm Clock Bakelike Green)
4,0.076190,(Party Bunting)
5,0.590476,(Postage)
6,0.238095,(Regency Cakestand 3 Tier)
7,0.104762,(Plasters In Tin Skulls)
8,0.104762,(Classic Metal Birdcage Plant Holder)
9,0.066667,(Plasters In Tin Circus Parade )


In [36]:
# Compute the association rules based on the frequent itemsets
from mlxtend.frequent_patterns import association_rules

In [37]:
# compute and print the association rules
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Plasters In Tin Skulls),(Postage),0.104762,0.590476,0.076190,0.727273,1.231672,0.014331,1.501587
1,(Classic Metal Birdcage Plant Holder),(Postage),0.104762,0.590476,0.085714,0.818182,1.385630,0.023855,2.252381
2,(Set Of 72 Retrospot Paper Doilies),(Postage),0.095238,0.590476,0.076190,0.800000,1.354839,0.019955,2.047619
3,(Poppy'S Playhouse Bedroom ),(Postage),0.076190,0.590476,0.066667,0.875000,1.481855,0.021678,3.276190
4,(Poppy'S Playhouse Kitchen),(Poppy'S Playhouse Bedroom ),0.066667,0.076190,0.066667,1.000000,13.125000,0.061587,inf
5,(Poppy'S Playhouse Bedroom ),(Poppy'S Playhouse Kitchen),0.076190,0.066667,0.066667,0.875000,13.125000,0.061587,7.466667
6,(6 Ribbons Rustic Charm),(Postage),0.142857,0.590476,0.123810,0.866667,1.467742,0.039456,3.071429
7,(Assorted Colour Bird Ornament),(Postage),0.114286,0.590476,0.104762,0.916667,1.552419,0.037279,4.914286
8,(Assorted Colour Bird Ornament),(6 Ribbons Rustic Charm),0.114286,0.142857,0.085714,0.750000,5.250000,0.069388,3.428571
9,"(Assorted Colour Bird Ornament, Postage)",(6 Ribbons Rustic Charm),0.104762,0.142857,0.085714,0.818182,5.727273,0.070748,4.714286


 <h4> Utilizando el ECLAT Basket Analisis

In [38]:
# you simply convert the transaction list into a dataframe
data = pd.DataFrame(transactions)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,115,116
0,Lunch Bag Suki Design,Lunch Bag Black Skull.,Lunch Bag Pink Polkadot,Lunch Bag Red Retrospot,Lunch Bag Cars Blue,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Folkart Zinc Heart Christmas Dec,Heart Decoration Painted Zinc,Wooden Tree Christmas Scandinavian,Lavender Scented Fabric Heart,Alarm Clock Bakelike Green,Blue Coat Rack Paris Fashion,Cabin Bag Vintage Retrospot,Party Bunting,Wooden Heart Christmas Scandinavian,Circus Parade Lunch Box,...,None,None,None,None,None,None,None,None,None,None
2,Welcome Wooden Block Letters,Home Building Block Word,Love Building Block Word,Classic Metal Birdcage Plant Holder,Postage,Gin & Tonic Diet Greeting Card,Regency Cakestand 3 Tier,White Skull Hot Water Bottle,Home Sweet Home Metal Sign,"Birthday Card, Retro Spot",...,None,None,None,None,None,None,None,None,None,None
3,Jumbo Bag Baroque Black White,Jumbo Bag Scandinavian Paisley,Jumbo Bag Red Retrospot,Skull Shoulder Bag,Party Food Shopper Bag,Jumbo Bag Pink Polkadot,Suki Shoulder Bag,Strawberry Shopper Bag,Jumbo Bag Woodland Animals,Red Retrospot Shopper Bag,...,None,None,None,None,None,None,None,None,None,None
4,Manual,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [40]:
# we are looking for itemSETS
# we do not want to have any individual products returned
min_n_products = 2

In [41]:
# we want to set min support to 7
# but we have to express it as a percentage
min_support = 7/len(transactions)

In [42]:
# we have no limit on the size of association rules
# so we set it to the longest transaction
max_length = max([len(x) for x in transactions])

In [43]:
max_length

117

In [44]:
from pyECLAT import ECLAT

In [45]:
# create an instance of eclat
my_eclat = ECLAT(data=data, verbose=True)

100%|█████████████████████████████████████████████████████████████████████████████| 1108/1108 [00:01<00:00, 590.93it/s]


In [ ]:
# fit the algorithm
rule_indices, rule_supports = my_eclat.fit(min_support=min_support,
                                           min_combination=min_n_products,
                                           max_combination=max_length)

0it [00:00, ?it/s]

Combination 2 by 2


780it [05:25,  2.40it/s]
0it [00:00, ?it/s]

Combination 3 by 3


3238it [19:08,  3.77it/s]

In [ ]:
print(rule_supports)